In [1]:
import networkx as nx
import numpy as np
from pyspark.sql import SparkSession

from snpp.cores.lowrank import alq_spark
from snpp.utils.matrix import split_train_test, load_sparse_csr
from snpp.utils.signed_graph import fill_diagonal

In [2]:
dataset = 'slashdot'
lambda_ = 0.2
k = 40
max_iter = 100
random_seed = 123456

In [3]:
sc.setCheckpointDir('.checkpoint')  # stackoverflow errors
    
    
m = load_sparse_csr('data/{}.npz'.format(dataset))
train_m, test_m = split_train_test(m, [.9, .1])

train_m = fill_diagonal(train_m)
targets = list(zip(*test_m.nonzero()))



100%|██████████| 77357/77357 [00:01<00:00, 74282.57it/s]


In [4]:
print(train_m.shape)
print(test_m.shape)
print(train_m[0, 0])

train_m = fill_diagonal(train_m)
print(train_m[0, 0])


(77357, 77357)
(77357, 77357)
1.0


100%|██████████| 77357/77357 [00:01<00:00, 46470.36it/s]


1.0


In [5]:
X, Y = alq_spark(train_m, k=k, sc=sc,
                 lambda_=lambda_, iterations=max_iter,
                 seed=random_seed)



KeyboardInterrupt: 

In [ ]:
print(X.shape)
print(m.shape)

In [ ]:
def predict_signs(X, Y, targets, sc):
    Xb, Yb = sc.broadcast(X), sc.broadcast(np.transpose(Y))
    preds = sc.parallelize(targets).map(
        lambda e: (e[0], e[1], np.sign(np.dot(Xb.value[e[0]], Yb.value[e[1]])))
    ).collect()
    return preds

In [ ]:
preds = predict_signs(X, Y, targets, sc)

In [ ]:
test_m = test_m.todok()
truth = set((i, j, test_m[i, j]) for i, j in targets)
assert len(truth) == len(preds)
print('=> final accuracy {} (original lowrank)'.format(len(truth.intersection(preds)) / len(truth)))

In [ ]:
### predict using symmetric matrix
g, test_m = load_train_test_data(dataset, False)

targets = list(zip(*test_m.nonzero()))

In [ ]:
A = nx.to_scipy_sparse_matrix(g, nodelist=g.nodes(),
                              weight='sign', format='csr')
A = fill_diagonal(A)
# assert (A.nnz - A.shape[0]) == len(targets)               

In [ ]:
print(A.shape)

In [ ]:
X, Y = alq_spark(A, k=k, sc=sc,
                 lambda_=lambda_, iterations=max_iter,
                 seed=random_seed)

In [ ]:
test_m = test_m.todok()
truth = set((i, j, test_m[i, j]) for i, j in targets)
preds = predict_signs(X, Y, targets)

assert len(truth) == len(preds)
assert set((i, j) for i, j, _ in predict_by_undirectionality.ipynbs) == set((i ,j) for i, j, _ in truth)

print('=> final accuracy {} (using symmetric graph)'.format(len(truth.intersection(preds)) / len(truth)))